In [2]:
from argparse import Namespace
import time
from tokenize import group
from typing import Dict, Any
from torch.utils.data import Dataset
import torch
from torchvision import transforms, datasets
import numpy as np
import json
import pandas as pd
import os
from PIL import Image
import shutil
from typing import Tuple

/home/sangyoonyu/workspace/fair-fl-learning/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
os.mkdir('../data/celebA/celeba-resized/img_align_celeba')

In [17]:
for imgfile in os.listdir('../data/celebA/celeba/img_align_celeba'):
    im = Image.open('../data/celebA/celeba/img_align_celeba/' + imgfile)
    im = im.resize((64, 64))
    im.save('../data/celebA/celeba-resized/img_align_celeba/' + imgfile)

In [13]:
# os.mkdir('../data/celebA/celeba-resized/train')
# os.mkdir('../data/celebA/celeba-resized/val')
# os.mkdir('../data/celebA/celeba-resized/test')

In [6]:
partition_df = pd.read_csv('../data/celebA/celeba/list_eval_partition.csv')
attr_df = pd.read_csv('../data/celebA/celeba/list_attr_celeba.csv')
partition_df.join(attr_df.set_index('image_id'), on='image_id')

,image_id,partition,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000001.jpg,0,-1,1,1,-1,-1,-1,-1,-1,...,-1,1,1,-1,1,-1,1,-1,-1,1
1,000002.jpg,0,-1,-1,-1,1,-1,-1,-1,1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
2,000003.jpg,0,-1,-1,-1,-1,-1,-1,1,-1,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
3,000004.jpg,0,-1,-1,1,-1,-1,-1,-1,-1,...,-1,-1,1,-1,1,-1,1,1,-1,1
4,000005.jpg,0,-1,1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202594,202595.jpg,2,-1,-1,1,-1,-1,-1,1,-1,...,-1,-1,-1,-1,-1,-1,1,-1,-1,1
202595,202596.jpg,2,-1,-1,-1,-1,-1,1,1,-1,...,-1,1,1,-1,-1,-1,-1,-1,-1,1
202596,202597.jpg,2,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
202597,202598.jpg,2,-1,1,1,-1,-1,-1,1,-1,...,-1,1,-1,1,1,-1,1,-1,-1,1


In [15]:
train = []
val = []
test = []
for index, row in df.iterrows():
    if row['partition'] == 0:
        train.append(row['image_id'])
    elif row['partition'] == 1:
        val.append(row['image_id'])
    elif row['partition'] == 2:
        test.append(row['image_id'])

In [19]:
original_dir = '/home/ysy970923/workspace/fair-fl-adversarial-learning/data/celebA/celeba-resized/img_align_celeba'
move_dir = '/home/ysy970923/workspace/fair-fl-adversarial-learning/data/celebA/celeba-resized'

for file in train:
    shutil.move(original_dir + '/' + file, move_dir + '/train/' + file)

for file in val:
    shutil.move(original_dir + '/' + file, move_dir + '/val/' + file)

for file in test:
    shutil.move(original_dir + '/' + file, move_dir + '/test/' + file)

In [88]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/list_attr_celeba.csv')
partition_df = pd.read_csv('../data/celebA/celeba-resized/list_eval_partition.csv')

merge_df = attr_df.merge(partition_df, left_on='image_id', right_on='image_id')

grouped = merge_df.groupby(merge_df.partition)

train_df = grouped.get_group(0)
val_df = grouped.get_group(1)
test_df = grouped.get_group(2)

train_df.to_csv('../data/celebA/celeba-resized/train_attr_list.csv')
val_df.to_csv('../data/celebA/celeba-resized/val_attr_list.csv')
test_df.to_csv('../data/celebA/celeba-resized/test_attr_list.csv')

# train_set = set(train)
# val_set = set(val)
# test_set = set(test)

# train_df = pd.DataFrame()
# val_df = pd.DataFrame()
# test_df = pd.DataFrame()

# for index, row in attr_df.iterrows():
#     if row['image_id'] in train_set:
#         train_set.remove(row['image_id'])
#         train_df.append(row)
#     elif row['image_id'] in val_set:
#         val_set.remove(row['image_id'])
#         val_df.append(row)
#     elif row['image_id'] in test_set:
#         test_set.remove(row['image_id'])
#         test_df.append(row)

In [3]:
import pandas as pd
pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv').columns

Index(['Unnamed: 0', 'image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows',
       'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips',
       'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair',
       'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee',
       'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male',
       'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie', 'Young', 'partition'],
      dtype='object')

In [94]:
for i , x in df.iterrows():
    print(dict(x))
    break

{'image_id': '000001.jpg', '5_o_Clock_Shadow': -1, 'Arched_Eyebrows': 1, 'Attractive': 1, 'Bags_Under_Eyes': -1, 'Bald': -1, 'Bangs': -1, 'Big_Lips': -1, 'Big_Nose': -1, 'Black_Hair': -1, 'Blond_Hair': -1, 'Blurry': -1, 'Brown_Hair': 1, 'Bushy_Eyebrows': -1, 'Chubby': -1, 'Double_Chin': -1, 'Eyeglasses': -1, 'Goatee': -1, 'Gray_Hair': -1, 'Heavy_Makeup': 1, 'High_Cheekbones': 1, 'Male': -1, 'Mouth_Slightly_Open': 1, 'Mustache': -1, 'Narrow_Eyes': -1, 'No_Beard': 1, 'Oval_Face': -1, 'Pale_Skin': -1, 'Pointy_Nose': 1, 'Receding_Hairline': -1, 'Rosy_Cheeks': -1, 'Sideburns': -1, 'Smiling': 1, 'Straight_Hair': 1, 'Wavy_Hair': -1, 'Wearing_Earrings': 1, 'Wearing_Hat': -1, 'Wearing_Lipstick': 1, 'Wearing_Necklace': -1, 'Wearing_Necktie': -1, 'Young': 1}


In [36]:
class CelebADataset(Dataset):
    def __init__(self, dir: str, target_column: str, transform=None, split: str = 'train'):
        super(CelebADataset, self).__init__()
        self.dir = dir + '/{}/'.format(split)
        samples = []
        attr_df = pd.read_csv(dir + '/' + split + '_attr_list.csv')
        possible_targets = np.sort(attr_df['Attractive'].unique())
        print(possible_targets)
        for _, row in attr_df.iterrows():
            image_id = row['image_id']
            target = np.where(possible_targets == row[target_column])[0][0]
            samples.append((image_id, target))
        self.samples = samples
        self.transform = transform

    def __getitem__(self, index: int) -> Tuple[Any, Any]:
        path, target = self.samples[index]
        with open(self.dir + path, 'rb') as f:
            img = Image.open(f)
            sample = img.convert('RGB')
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

    def __len__(self) -> int:
        return len(self.samples)

In [5]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/test_attr_list.csv')
subgroup_keys = attr_df['Male'].unique()
subgroups = attr_df.groupby(['Male'])
{k: np.where(attr_df['Male'] == k) for k in subgroup_keys}

{0: (array([    0,     2,     4, ..., 39824, 39827, 39828]),),
 1: (array([    1,     3,     7, ..., 39819, 39825, 39826]),)}

In [11]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/test_attr_list.csv')
attr_df['Attractive'].unique()

array([-1,  1])

In [3]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv')
attr_df

,image_id,5_o_Clock_Shadow,Arched_Eyebrows,Attractive,Bags_Under_Eyes,Bald,Bangs,Big_Lips,Big_Nose,Black_Hair,...,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young,partition
0,000001.jpg,0,1,1,0,0,0,0,0,0,...,1,1,0,1,0,1,0,0,1,0
1,000002.jpg,0,0,0,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,1,0
2,000003.jpg,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,0
3,000004.jpg,0,0,1,0,0,0,0,0,0,...,0,1,0,1,0,1,1,0,1,0
4,000005.jpg,0,1,1,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162765,162766.jpg,0,0,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
162766,162767.jpg,0,0,0,1,0,0,0,1,0,...,1,0,1,0,0,0,0,0,0,0
162767,162768.jpg,1,0,1,0,0,0,0,0,1,...,1,1,0,0,0,0,0,1,1,0
162768,162769.jpg,0,0,0,0,0,0,1,1,1,...,1,0,0,1,0,0,0,0,1,0


In [4]:
# attr_df = pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv')
# attr_df = attr_df.where(attr_df != 1, 1)
# attr_df = attr_df.where(attr_df != -1, 0)
# attr_df = attr_df.drop(columns=['Unnamed: 0'])
# attr_df.to_csv('../data/celebA/celeba-resized/train_attr_list.csv', index=False)

# attr_df = pd.read_csv('../data/celebA/celeba-resized/test_attr_list.csv')
# attr_df = attr_df.where(attr_df != 1, 1)
# attr_df = attr_df.where(attr_df != -1, 0)
# attr_df = attr_df.drop(columns=['Unnamed: 0'])
# attr_df.to_csv('../data/celebA/celeba-resized/test_attr_list.csv', index=False)

attr_df = pd.read_csv('../data/celebA/celeba-resized/val_attr_list.csv')
attr_df = attr_df.where(attr_df != 1, 1)
attr_df = attr_df.where(attr_df != -1, 0)
attr_df = attr_df.drop(columns=['Unnamed: 0'])
attr_df.to_csv('../data/celebA/celeba-resized/val_attr_list.csv', index=False)

KeyError: "['Unnamed: 0'] not found in axis"

In [47]:
test_df = pd.read_csv('../data/celebA/celeba-resized/test_attr_list.csv')
val_df = pd.read_csv('../data/celebA/celeba-resized/val_attr_list.csv')
attr_df = pd.concat([val_df, test_df])
attr_df.to_csv('../data/celebA/celeba-resized/test_attr_list.csv', index=False)

In [2]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv')
target_column = 'Blond_Hair'
protected_variable = 'Male'
# group_00 = attr_df.loc[(attr_df[target_column] == 0) & (attr_df[protected_variable] == 0)]
# group_01 = attr_df.loc[(attr_df[target_column] == 0) & (attr_df[protected_variable] == 1)]
# group_10 = attr_df.loc[(attr_df[target_column] == 1) & (attr_df[protected_variable] == 0)]
# group_11 = attr_df.loc[(attr_df[target_column] == 1) & (attr_df[protected_variable] == 1)]

# if len(group_00) <= len(group_10):
#     group_00 = group_00.sample(n=min(len(group_10)//8, len(group_00)))
# else:
#     group_10 = group_10.sample(n=min(len(group_00)//8, len(group_10)))

# if len(group_01) <= len(group_11):
#     group_01 = group_01.sample(n=min(len(group_11)//8, len(group_01)))
# else:
#     group_11 = group_11.sample(n=min(len(group_01)//8, len(group_11)))

# attr_df = pd.concat([group_00, group_01, group_10, group_11])

In [29]:
g_000 = torch.tensor((attr_df['Black_Hair'] == 0) & (attr_df['Male'] == 0) & (attr_df['Smiling'] == 0)).count_nonzero()
g_010 = torch.tensor((attr_df['Black_Hair'] == 0) & (attr_df['Male'] == 1) & (attr_df['Smiling'] == 0)).count_nonzero()
g_100 = torch.tensor((attr_df['Black_Hair'] == 1) & (attr_df['Male'] == 0) & (attr_df['Smiling'] == 0)).count_nonzero()
g_110 = torch.tensor((attr_df['Black_Hair'] == 1) & (attr_df['Male'] == 1) & (attr_df['Smiling'] == 0)).count_nonzero()
g_001 = torch.tensor((attr_df['Black_Hair'] == 0) & (attr_df['Male'] == 0) & (attr_df['Smiling'] == 1)).count_nonzero()
g_011 = torch.tensor((attr_df['Black_Hair'] == 0) & (attr_df['Male'] == 1) & (attr_df['Smiling'] == 1)).count_nonzero()
g_101 = torch.tensor((attr_df['Black_Hair'] == 1) & (attr_df['Male'] == 0) & (attr_df['Smiling'] == 1)).count_nonzero()
g_111 = torch.tensor((attr_df['Black_Hair'] == 1) & (attr_df['Male'] == 1) & (attr_df['Smiling'] == 1)).count_nonzero()
print(g_000, g_010, g_100, g_110, g_001, g_011, g_101, g_111)

tensor(35346) tensor(29081) tensor(8342) tensor(11921) tensor(40379) tensor(19058) tensor(10442) tensor(8201)


In [5]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv')
image_ids = attr_df['image_id'].to_list()
protected_variable1 = torch.tensor(attr_df['Black_Hair'])
protected_variable2 = torch.tensor(attr_df['Male'])
target = torch.tensor(attr_df['Smiling'])
bias_level = 4
samples = torch.zeros((2,2,2), dtype=torch.long)
for i in range(2):
    for j in range(2):
        for k in range(2):
            samples[i][j][k] = ((protected_variable1 == i) & (protected_variable2 == j) & (target == k)).count_nonzero()
print(samples)

bias = torch.tensor([[[1,1], [1,1]], [[1,1], [1,1]]], dtype=torch.float)
if ((protected_variable1 == 0) & (target == 0)).count_nonzero() > ((protected_variable1 == 1) & (target == 0)).count_nonzero():
    bias[0][0][0] *= bias_level
    bias[0][1][0] *= bias_level
    bias[1][0][1] *= bias_level
    bias[1][1][1] *= bias_level

else:
    bias[0][0][1] *= bias_level
    bias[0][1][1] *= bias_level
    bias[1][0][0] *= bias_level
    bias[1][1][0] *= bias_level

if ((protected_variable2 == 0) & (target == 0)).count_nonzero() > ((protected_variable2 == 1) & (target == 0)).count_nonzero():
    bias[0][0][0] *= bias_level
    bias[1][0][0] *= bias_level
    bias[0][1][1] *= bias_level
    bias[1][1][1] *= bias_level
else:
    bias[0][0][1] *= bias_level
    bias[1][0][1] *= bias_level
    bias[0][1][0] *= bias_level
    bias[1][1][0] *= bias_level
print(bias)
x = samples / bias

unit = x.min()
new_size = bias * unit
new_indices = []
new_size = new_size.type(torch.LongTensor)
print(new_size)
for i in range(2):
    for j in range(2):
        for k in range(2):
            indices = ((protected_variable1 == i) & (protected_variable2 == j) & (target == k)).nonzero()
            indices = indices.index_select(dim=0, index = torch.randperm(len(indices)))[:new_size[i][j][k]]
            new_indices.append(indices.flatten())
new_indices = torch.concat(new_indices)
protected_variable1 = protected_variable1[new_indices]
protected_variable2 = protected_variable2[new_indices]
target = target[new_indices]

samples = torch.zeros((2,2,2), dtype=torch.long)
for i in range(2):
    for j in range(2):
        for k in range(2):
            samples[i][j][k] = ((protected_variable1 == i) & (protected_variable2 == j) & (target == k)).count_nonzero()
print(samples)
image_ids = [image_ids[i] for i in new_indices]

tensor([[[35346, 40379],
         [29081, 19058]],

        [[ 8342, 10442],
         [11921,  8201]]])
tensor([[[16.,  1.],
         [ 4.,  4.]],

        [[ 4.,  4.],
         [ 1., 16.]]])
tensor([[[8201,  512],
         [2050, 2050]],

        [[2050, 2050],
         [ 512, 8201]]])
tensor([[[8201,  512],
         [2050, 2050]],

        [[2050, 2050],
         [ 512, 8201]]])


In [7]:
len(attr_df)
len(new_indices)

NameError: name 'new_indices' is not defined

In [10]:
attr_df = pd.read_csv('../data/celebA/celeba-resized/train_attr_list.csv')
n_sample = len(attr_df.loc[(attr_df['Wavy_Hair'] == 1) & (attr_df['Male'] == 0)])
print(f'smiling woman: {n_sample}')
n_sample = len(attr_df.loc[(attr_df['Wavy_Hair'] == 1) & (attr_df['Male'] == 1)])
print(f'smiling man: {n_sample}')
n_sample = len(attr_df.loc[(attr_df['Wavy_Hair'] == 0) & (attr_df['Male'] == 0)])
print(f'not smiling woman: {n_sample}')
n_sample = len(attr_df.loc[(attr_df['Wavy_Hair'] == 0) & (attr_df['Male'] == 1)])
print(f'not smiling man: {n_sample}')

smiling woman: 42220
smiling man: 9762
not smiling woman: 52289
not smiling man: 58499


In [41]:
np.array(attr_df['Smiling'])

array([1, 1, 0, ..., 1, 1, 0])

In [25]:
def get_subgroups(split = 'train', protected_variable = 'Male'):
    attr_df = pd.read_csv('../data/celebA/celeba-resized/' + split + '_attr_list.csv')
    subgroup_keys = attr_df[protected_variable].unique()
    subgroups = {k: np.where(attr_df[protected_variable] == k)[0]  for k in subgroup_keys}
    return subgroups

get_subgroups()


{-1: array([     0,      1,      3, ..., 162759, 162760, 162769]),
 1: array([     2,      6,      7, ..., 162766, 162767, 162768])}

In [37]:
dir = '../data/celebA/celeba-resized'
df = pd.read_csv('../data/celebA/celeba-resized/list_attr_celeba.csv')

apply_transform_train = transforms.Compose(
    [transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                            (0.5, 0.5, 0.5)),
                            ]
)

train_dataset = CelebADataset(dir, 'Attractive', transform=apply_transform_train, split='train')

[-1  1]


In [29]:
train_dataset.samples[2]

('182640.jpg', 1)

In [27]:
dir = '../data/celebA'
apply_transform_train = transforms.Compose(
    [transforms.Resize((64,64)),
    transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                            (0.5, 0.5, 0.5)),
                            ]
)
apply_transform_test = transforms.Compose(
    [transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5),
                            (0.5, 0.5, 0.5)),
                            ]
)
train_dataset = datasets.ImageFolder(dir + '/celeba-resized', transform=apply_transform_train)

In [40]:
train_dataset[0]

(tensor([[[ 1.0000,  1.0000,  0.9608,  ...,  0.8824,  0.9137,  0.9843],
          [ 0.9922,  0.9216,  0.9608,  ...,  0.9686,  0.9451,  0.8980],
          [ 1.0000,  0.9451,  0.9765,  ...,  0.9686,  1.0000,  0.9765],
          ...,
          [-0.9608, -0.9922, -0.9843,  ..., -0.9843, -0.9765, -1.0000],
          [-0.9922, -1.0000, -0.9922,  ..., -1.0000, -0.9922, -1.0000],
          [-1.0000, -0.9922, -0.9529,  ..., -1.0000, -0.9843, -1.0000]],
 
         [[ 1.0000,  1.0000,  0.9686,  ...,  0.8275,  0.8588,  0.9294],
          [ 1.0000,  0.9294,  0.9686,  ...,  0.9137,  0.8902,  0.8431],
          [ 1.0000,  0.9529,  0.9843,  ...,  0.9137,  0.9529,  0.9216],
          ...,
          [-0.9451, -0.9765, -0.9843,  ..., -0.9843, -0.9765, -1.0000],
          [-0.9922, -1.0000, -1.0000,  ..., -1.0000, -0.9922, -1.0000],
          [-1.0000, -0.9922, -0.9686,  ..., -1.0000, -0.9843, -1.0000]],
 
         [[ 0.9686,  0.9686,  0.9216,  ...,  0.7804,  0.8118,  0.8824],
          [ 0.9608,  0.8902,